# 05b - Vertex AI > Training > Custom Jobs - With Python Source Distribution

### 05 Series Overview
Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

In the `05` notebook, the model training happened directly in the notebook.  The models were then imported to Vertex AI and deployed to an endpoint for online predictions. 

In this `05a-05i` series of demonstrations, the same model is trained using managed computing resources in Vertex AI as custom training jobs.  These jobs will be demonstrated as:

-  Custom Job from a python script (`05a`), python source distribution (`05b`), and custom container (`05c`)
-  Training Pipeline that trains and saves models from a python script (`05d`), python source distribution (`05e`), and custom container (`05f`)
-  Hyperparameter Tuning Jobs from a python script (`05g`), python source distribution (`05h`), and custom container (`05i`)

### This Notebook (`05b`): An extension of `05a` that saves the script within a source distribution
This notebook trains the same Tensorflow Keras model from `05` by first modifying and saving the training code to a python script (same as `05a`).  Then a Python source distribution is built containing the script.  While this example fits nicely in a single script, larger examples will benefit from the flexibility offered by source distributions and this notebook gives an example of making the shift.  

The source distribution is then used as an input for a Vertex AI > Training > Custom Job that is also assigned compute resources and a [pre-built container for custom training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) for executing the training in a managed service.  This is done with the [Vertex AI Python SDK](https://googleapis.dev/python/aiplatform/latest/aiplatform.html#) using the class [`aiplatform.CustomJob()`](https://googleapis.dev/python/aiplatform/latest/aiplatform.html#google.cloud.aiplatform.CustomJob).

The training can be reviewed with Vertex AI's managed Tensorboard under Vertex AI > Experiments > Experiments, or by clicking on the `05b...` custom job under Vertex AI > Training > Custom Jobs and then clicking the 'Open Tensorboard' link.

<img src="architectures/overview/Training.png">

### Prerequisites:
-  01 - BigQuery - Table Data Source
-  Understanding:
    -  05 - Vertex AI > Notebooks - Models Built in Notebooks with Tensorflow
        -  Contains a more granular review of the Tensorflow model training

### Resources:
-  [BigQuery Tensorflow Reader](https://www.tensorflow.org/io/tutorials/bigquery)
-  [Keras Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)
   -  [Keras API](https://www.tensorflow.org/api_docs/python/tf/keras)
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Tensorflow Python Client](https://www.tensorflow.org/api_docs/python/tf)
-  [Tensorflow I/O Python Client](https://www.tensorflow.org/io/api_docs/python/tfio/bigquery)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)
-  [Create a Python source distribution](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container) for a Vertex AI custom training job
-  Containers for training (Pre-Built)
   -  [Overview](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)
   -  [List](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)



-----
## Vertex AI - Conceptual Flow

<img src="architectures/slides/05b_arch.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/05b_console.png">

---
## Setup

inputs:

In [39]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [40]:
REGION = 'us-central1'
DATANAME = 'fraud'
NOTEBOOK = '05b'
SERIES = '05'

# Resources
TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-7:latest'
DEPLOY_IMAGE ='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest'
TRAIN_COMPUTE = 'n1-standard-4'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters
EPOCHS = 10
BATCH_SIZE = 100

packages:

In [41]:
from google.cloud import aiplatform
from datetime import datetime

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [42]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [43]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
DIR = f"temp/{NOTEBOOK}"

In [44]:
# Give service account roles/storage.objectAdmin permissions
# Console > IMA > Select Account <projectnumber>-compute@developer.gserviceaccount.com > edit - give role
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

environment:

In [45]:
!rm -rf {DIR}
!mkdir -p {DIR}

Experiment Tracking:

In [46]:
TASK = 'classification'
MODEL_TYPE = 'dnn'
EXPERIMENT_NAME = f'experiment-{SERIES}-{NOTEBOOK}-{TASK}-{MODEL_TYPE}'
RUN_NAME = f'run-{TIMESTAMP}'

---
## Get Vertex AI Experiments Tensorboard Instance Name
[Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview) has managed [Tensorboard](https://www.tensorflow.org/tensorboard) instances that you can track Tensorboard Experiments (a training run or hyperparameter tuning sweep).  

The training job will show up as an experiment for the Tensorboard instance and have the same name as the training job ID.

This code checks to see if a Tensorboard Instance has been created in the project, retrieves it if so, creates it otherwise:

In [47]:
tb = aiplatform.Tensorboard.list(filter=f"labels.series={SERIES}")
if tb:
    tb = tb[0]
else: 
    tb = aiplatform.Tensorboard.create(display_name = SERIES, labels = {'series' : f'{SERIES}'})

In [48]:
tb.resource_name

'projects/1026793852137/locations/us-central1/tensorboards/3514619704511561728'

---
## Setup Vertex AI Experiments

The code in this section initializes the experiment and starts a run that represents this notebook.  Throughout the notebook sections for model training and evaluation information will be logged to the experiment using:
- [.log_params](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_params)
- [.log_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_metrics)
- [.log_time_series_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_time_series_metrics)

In [49]:
aiplatform.init(experiment = EXPERIMENT_NAME, experiment_tensorboard = tb.resource_name)

---
## Training

### Assemble Python File for Training

Create the main python trainer file as `/train.py`:

In [50]:
!mkdir -p {DIR}/source/trainer

In [59]:
%%writefile {DIR}/source/trainer/train.py

# package import
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
import tensorflow as tf
from google.cloud import bigquery
from google.cloud import aiplatform
import argparse
import os
import sys

# import argument to local variables
parser = argparse.ArgumentParser()
# the passed param, dest: a name for the param, default: if absent fetch this param from the OS, type: type to convert to, help: description of argument
parser.add_argument('--epochs', dest = 'epochs', default = 10, type = int, help = 'Number of Epochs')
parser.add_argument('--batch_size', dest = 'batch_size', default = 32, type = int, help = 'Batch Size')
parser.add_argument('--var_target', dest = 'var_target', type=str)
parser.add_argument('--var_omit', dest = 'var_omit', type=str, nargs='*')
parser.add_argument('--project_id', dest = 'project_id', type=str)
parser.add_argument('--dataname', dest = 'dataname', type=str)
parser.add_argument('--region', dest = 'region', type=str)
parser.add_argument('--notebook', dest = 'notebook', type=str)
parser.add_argument('--series', dest = 'series', type=str)
parser.add_argument('--experiment_name', dest = 'experiment_name', type=str)
parser.add_argument('--run_name', dest = 'run_name', type=str)
args = parser.parse_args()

# clients
bigquery = bigquery.Client(project = args.project_id)
aiplatform.init(project = args.project_id, location = args.region)

# Vertex AI Experiment
expRun = aiplatform.ExperimentRun.create(run_name = args.run_name, experiment = args.experiment_name)
expRun.log_params({'DATANAME': args.dataname, 'NOTEBOOK': args.notebook, 'SERIES': args.series, 'PROJECT_ID': args.project_id, 'VAR_TARGET': args.var_target})

# get schema from bigquery source
query = f"SELECT * FROM {args.dataname}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{args.dataname}_prepped'"
schema = bigquery.query(query).to_dataframe()

# get number of classes from bigquery source
nclasses = bigquery.query(query = f'SELECT DISTINCT {args.var_target} FROM {args.dataname}.{args.dataname}_prepped WHERE {args.var_target} is not null').to_dataframe()
nclasses = nclasses.shape[0]
expRun.log_params({'data_source': f'bq://{args.dataname}.{args.dataname}_prepped', 'nclasses': nclasses, 'var_split': 'splits'})

# Make a list of columns to omit
OMIT = args.var_omit + ['splits']

# use schema to prepare a list of columns to read from BigQuery
selected_fields = schema[~schema.column_name.isin(OMIT)].column_name.tolist()

# all the columns in this data source are either float64 or int64
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in schema[~schema.column_name.isin(OMIT)].data_type.tolist()]

# remap input data to Tensorflow inputs of features and target
def transTable(row_dict):
    target = row_dict.pop(args.var_target)
    target = tf.one_hot(tf.cast(target, tf.int64), nclasses)
    target = tf.cast(target, tf.float32)
    return(row_dict, target)

# function to setup a bigquery reader with Tensorflow I/O
def bq_reader(split):
    reader = BigQueryClient()

    training = reader.read_session(
        parent = f"projects/{args.project_id}",
        project_id = args.project_id,
        table_id = f"{args.dataname}_prepped",
        dataset_id = args.dataname,
        selected_fields = selected_fields,
        output_types = output_types,
        row_restriction = f"splits='{split}'",
        requested_streams = 3
    )
    
    return training

# setup feed for train, validate and test
train = bq_reader('TRAIN').parallel_read_rows().prefetch(1).map(transTable).shuffle(args.batch_size*10).batch(args.batch_size)
validate = bq_reader('VALIDATE').parallel_read_rows().prefetch(1).map(transTable).batch(args.batch_size)
test = bq_reader('TEST').parallel_read_rows().prefetch(1).map(transTable).batch(args.batch_size)
expRun.log_params({'BATCH_SIZE': args.batch_size, 'SHUFFLE': 10*args.batch_size, 'PREFETCH': 1})

# Logistic Regression

# model input definitions
feature_columns = {header: tf.feature_column.numeric_column(header) for header in selected_fields if header != args.var_target}
feature_layer_inputs = {header: tf.keras.layers.Input(shape = (1,), name = header) for header in selected_fields if header != args.var_target}

# feature columns to a Dense Feature Layer
feature_layer_outputs = tf.keras.layers.DenseFeatures(feature_columns.values(), name = 'feature_layer')(feature_layer_inputs)

# batch normalization then Dense with softmax activation to nclasses
layers = tf.keras.layers.BatchNormalization(name = 'batch_normalization_layer')(feature_layer_outputs)
layers = tf.keras.layers.Dense(64, activation = 'relu', name = 'hidden_layer')(layers)
layers = tf.keras.layers.Dense(32, activation = 'relu', name = 'embedding_layer')(layers)
layers = tf.keras.layers.Dense(nclasses, activation = tf.nn.softmax, name = 'prediction_layer')(layers)

# the model
model = tf.keras.Model(
    inputs = feature_layer_inputs,
    outputs = layers,
    name = args.dataname
)
opt = tf.keras.optimizers.SGD() #SGD or Adam
loss = tf.keras.losses.CategoricalCrossentropy()
model.compile(
    optimizer = opt,
    loss = loss,
    metrics = ['accuracy', tf.keras.metrics.AUC(curve='PR', name = 'auprc')]
)

# setup tensorboard logs and train
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'], histogram_freq=1)
history = model.fit(train, epochs = args.epochs, callbacks = [tensorboard_callback], validation_data = validate)
expRun.log_params({'epochs': history.params['epochs']})
for e in range(0, history.params['epochs']):
    expRun.log_time_series_metrics(
        {
            'train_loss': history.history['loss'][e],
            'train_accuracy': history.history['accuracy'][e],
            'train_auprc': history.history['auprc'][e],
            'val_loss': history.history['val_loss'][e],
            'val_accuracy': history.history['val_accuracy'][e],
            'val_auprc': history.history['val_auprc'][e]
        }
    )

# evaluations:
loss, accuracy, auprc = model.evaluate(test)
expRun.log_metrics({'test_loss': loss, 'test_accuracy': accuracy, 'test_auprc': auprc})
loss, accuracy, auprc = model.evaluate(validate)
expRun.log_metrics({'val_loss': loss, 'val_accuracy': accuracy, 'val_auprc': auprc})
loss, accuracy, auprc = model.evaluate(train)
expRun.log_metrics({'train_loss': loss, 'train_accuracy': accuracy, 'train_auprc': auprc})

# output the model save files
model.save(os.getenv("AIP_MODEL_DIR"))
expRun.log_params({'MODEL_URI': os.getenv("AIP_MODEL_DIR")})
expRun.end_run()

Writing temp/05b/source/trainer/train.py


### Assemble Python Source Distribution

create `setup.py` file:

In [60]:
setupfile = f"""from setuptools import setup
from setuptools import find_packages

REQUIRED_PACKAGES = ['tensorflow_io', 'google-cloud-aiplatform>={aiplatform.__version__}']

setup(
    name = 'trainer',
    version = '0.1',
    install_requires = REQUIRED_PACKAGES, 
    packages = find_packages(),
    include_package_data = True,
    description='Training Package'
)
"""
with open(f'{DIR}/source/setup.py', 'w') as f:
    f.write(setupfile)

add `__init__.py` file to the trainer modules folder:

In [61]:
!touch {DIR}/source/trainer/__init__.py

Create the source distribution and copy it to the projects storage bucket:
- change to the local direcory with the source folder
- remove any previous distributions
- tar and gzip the source folder
- copy the distribution to the project folder on GCS
- change back to the local project directory

In [62]:
%cd {DIR}

!rm -f source.tar source.tar.gz
!tar cvf source.tar source
!gzip source.tar
!gsutil cp source.tar.gz {URI}/{TIMESTAMP}/source.tar.gz

temp = '../'*(DIR.count('/')+1)
%cd {temp}

/home/jupyter/vertex-ai-mlops/temp/05b
source/
source/setup.py
source/trainer/
source/trainer/__init__.py
source/trainer/train.py
Copying file://source.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  2.6 KiB/  2.6 KiB]                                                
Operation completed over 1 objects/2.6 KiB.                                      
/home/jupyter/vertex-ai-mlops


### Setup Training Job

In [63]:
CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--batch_size=" + str(BATCH_SIZE),
    "--var_target=" + VAR_TARGET,
    "--var_omit=" + VAR_OMIT,
    "--project_id=" + PROJECT_ID,
    "--dataname=" + DATANAME,
    "--region=" + REGION,
    "--notebook=" + NOTEBOOK,
    "--series=" + SERIES,
    "--experiment_name=" + EXPERIMENT_NAME,
    "--run_name=" + RUN_NAME
]

In [64]:
MACHINE_SPEC = {
    "machine_type": TRAIN_COMPUTE,
    "accelerator_count": 0
}

WORKER_POOL_SPEC = [
    {
        "replica_count": 1,
        "machine_spec": MACHINE_SPEC,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [f"{URI}/{TIMESTAMP}/source.tar.gz"],
            "python_module": "trainer.train",
            "args": CMDARGS
        }
    }
]

In [65]:
customJob = aiplatform.CustomJob(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    worker_pool_specs = WORKER_POOL_SPEC,
    base_output_dir = f"{URI}/{TIMESTAMP}",
    staging_bucket = f"{URI}/{TIMESTAMP}",
    labels = {'series' : f'{SERIES}', 'notebook' : f'{NOTEBOOK}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}
)

### Run Training Job

In [66]:
customJob.run(
    service_account = SERVICE_ACCOUNT,
    tensorboard = tb.resource_name
)

Creating CustomJob
CustomJob created. Resource name: projects/1026793852137/locations/us-central1/customJobs/6469171550456119296
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/1026793852137/locations/us-central1/customJobs/6469171550456119296')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/6469171550456119296?project=1026793852137
View Tensorboard:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+6469171550456119296
CustomJob projects/1026793852137/locations/us-central1/customJobs/6469171550456119296 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/1026793852137/locations/us-central1/customJobs/6469171550456119296 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/1026793852137/locations/us-central1/customJobs/6469171550456119296 current state:
JobState.JOB_STATE_PENDING


In [67]:
customJob.display_name

'05b_fraud_20220823160808'

In [68]:
customJob.resource_name

'projects/1026793852137/locations/us-central1/customJobs/6469171550456119296'

Create hyperlines to job and tensorboard here:

In [69]:
job_link = f"https://console.cloud.google.com/vertex-ai/locations/{REGION}/training/{customJob.resource_name.split('/')[-1]}/cpu?cloudshell=false&project={PROJECT_ID}"
board_link = f"https://{REGION}.tensorboard.googleusercontent.com/experiment/{tb.resource_name.replace('/', '+')}+experiments+{customJob.resource_name.split('/')[-1]}"

In [70]:
print(f'Review the Job here:\n{job_link}')
print(f'Review the TensorBoard From the Job here:\n{board_link}')

Review the Job here:
https://console.cloud.google.com/vertex-ai/locations/us-central1/training/6469171550456119296/cpu?cloudshell=false&project=statmike-mlops-349915
Review the TensorBoard From the Job here:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+6469171550456119296


---
## Serving

### Upload The Model

In [71]:
modelmatch = aiplatform.Model.list(filter = f'labels.series={SERIES} AND labels.notebook={NOTEBOOK}')
if modelmatch:
    print("Model Already in Registry:")
    if RUN_NAME in modelmatch[0].version_aliases:
        print("This version already loaded, no action taken.")
        model = aiplatform.Model(model_name = modelmatch[0].resource_name)
    else:
        print('Loading model as new default version.')
        model = aiplatform.Model.upload(
            display_name = f'{NOTEBOOK}_{DATANAME}',
            model_id = f'model_{NOTEBOOK}_{DATANAME}',
            parent_model =  modelmatch[0].resource_name,
            serving_container_image_uri = DEPLOY_IMAGE,
            artifact_uri = f"{URI}/{TIMESTAMP}/model",
            is_default_version = True,
            version_aliases = [RUN_NAME],
            version_description = RUN_NAME,
            labels = {'series' : f'{SERIES}', 'notebook' : f'{NOTEBOOK}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}        
        )
else:
    print('This is a new model, creating in model registry')
    model = aiplatform.Model.upload(
        display_name = f'{NOTEBOOK}_{DATANAME}',
        model_id = f'model_{NOTEBOOK}_{DATANAME}',
        serving_container_image_uri = DEPLOY_IMAGE,
        artifact_uri = f"{URI}/{TIMESTAMP}/model",
        is_default_version = True,
        version_aliases = [RUN_NAME],
        version_description = RUN_NAME,
        labels = {'series' : f'{SERIES}', 'notebook' : f'{NOTEBOOK}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}
    )    

Model Already in Registry:
Loading model as new default version.
Creating Model
Create Model backing LRO: projects/1026793852137/locations/us-central1/models/model_05b_fraud/operations/928942533533237248
Model created. Resource name: projects/1026793852137/locations/us-central1/models/7746099000100519936@3
To use this Model in another session:
model = aiplatform.Model('projects/1026793852137/locations/us-central1/models/7746099000100519936@3')


**Note** on Version Aliases:
>Expectation is a name starting with `a-z` that can include `[a-zA-Z0-9-]`

**Retrieve a Model Resource**

[Resource](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model)
```Python
model = aiplatform.Model(model_name = f'model_{NOTEBOOK}_{DATANAME}') # retrieves default version
model = aiplatform.Model(model_name = f'model_{NOTEBOOK}_{DATANAME}@time-{TIMESTAMP}') # retrieves specific version
model = aiplatform.Model(model_name = f'model_{NOTEBOOK}_{DATANAME}', version = f'time-{TIMESTAMP}') # retrieves specific version
```

### Vertex AI Experiment Update and Review

In [72]:
expRun = aiplatform.ExperimentRun(run_name = RUN_NAME, experiment = EXPERIMENT_NAME)

In [73]:
expRun.log_params({
    'model.display_name': model.display_name,
    'model.versioned_resource_name': model.versioned_resource_name,
    'customJobs.display_name': customJob.display_name,
    'customJobs.resource_name': customJob.resource_name,
    'customJobs.link': job_link,
    'customJobs.tensorboard': board_link
})

In [74]:
expRun.update_state(state = aiplatform.gapic.Execution.State.COMPLETE)

In [75]:
exp = aiplatform.Experiment(experiment_name = EXPERIMENT_NAME)

In [76]:
exp.get_data_frame()

,experiment_name,run_name,run_type,state,param.var_split,param.SHUFFLE,param.MODEL_URI,param.BATCH_SIZE,param.DATANAME,param.customJobs.link,...,metric.val_accuracy,metric.train_loss,metric.train_accuracy,metric.val_auprc,time_series_metric.train_loss,time_series_metric.val_loss,time_series_metric.train_auprc,time_series_metric.val_accuracy,time_series_metric.val_auprc,time_series_metric.train_accuracy
0,experiment-05-05b-classification-dnn,run-20220823160808,system.ExperimentRun,COMPLETE,splits,1000.0,gs://statmike-mlops-349915/fraud/models/05b/20...,100.0,fraud,https://console.cloud.google.com/vertex-ai/loc...,...,0.999186,0.003939,0.999386,0.999624,0.003248,0.005356,0.999681,0.999186,0.999624,0.999382
1,experiment-05-05b-classification-dnn,run-20220822002951,system.ExperimentRun,COMPLETE,splits,1000.0,gs://statmike-mlops-349915/fraud/models/05b/20...,100.0,fraud,https://console.cloud.google.com/vertex-ai/loc...,...,0.999221,0.004174,0.999386,0.999484,0.003506,0.005241,0.999697,0.999221,0.999484,0.999377
2,experiment-05-05b-classification-dnn,run-20220821211522,system.ExperimentRun,COMPLETE,splits,1000.0,gs://statmike-mlops-349915/fraud/models/05b/20...,100.0,fraud,https://console.cloud.google.com/vertex-ai/loc...,...,0.999186,0.003859,0.999434,0.999576,0.003520,0.004870,0.999708,0.999186,0.999576,0.999347


Review the Experiments TensorBoard to compare runs:

In [77]:
print(f"The Experiment TensorBoard Link:\nhttps://{REGION}.tensorboard.googleusercontent.com/experiment/{tb.resource_name.replace('/', '+')}+experiments+{exp.name}")

The Experiment TensorBoard Link:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+experiment-05-05b-classification-dnn


### Create An Endpoint

In [48]:
endpoints = aiplatform.Endpoint.list(filter = f"labels.series={SERIES}")
if endpoints:
    endpoint = endpoints[0]
    print(f"Endpoint Exists: {endpoints[0].resource_name}")
else:
    endpoint = aiplatform.Endpoint.create(
        display_name = f"{SERIES}_{DATANAME}",
        labels = {'notebook':f"{SERIES}"}    
    )
    print(f"Endpoint Created: {endpoint.resource_name}")

Endpoint Exists: projects/1026793852137/locations/us-central1/endpoints/1329608625143939072


In [49]:
endpoint.display_name

'05_fraud'

### Deploy Model To Endpoint

In [50]:
dmodels = endpoint.list_models()

check = 0
if dmodels:
    for dmodel in dmodels:
        if dmodel.model == model.resource_name and dmodel.model_version_id == model.version_id and dmodel.id in endpoint.traffic_split:
            print(f'This model (and version) already deployed with {endpoint.traffic_split[dmodel.id]}% of traffic')
            check = 1
    
if check == 0:
    print(f'Deploying model with 100% of traffic...')
    endpoint.deploy(
        model = model,
        deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}',
        traffic_percentage = 100,
        machine_type = DEPLOY_COMPUTE,
        min_replica_count = 1,
        max_replica_count = 1
    ) 

Deploying model with 100% of traffic...
Deploying Model projects/1026793852137/locations/us-central1/models/4453967672492687360 to Endpoint : projects/1026793852137/locations/us-central1/endpoints/1329608625143939072
Deploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/1329608625143939072/operations/1546485438296883200


/opt/conda/lib/python3.7/site-packages/proto/marshal/rules/enums.py:40: UserWarning: Unrecognized DeploymentResourcesType enum value: 3
  value=value,


Endpoint model deployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/1329608625143939072


### Remove Deployed Models without Traffic

In [51]:
for dmodel in endpoint.list_models():
    if dmodel.id in endpoint.traffic_split:
        print(f"Model {dmodel.display_name} has traffic = {endpoint.traffic_split[dmodel.id]}")
    else:
        endpoint.undeploy(deployed_model_id = dmodel.id)
        print(f"Undeployed {dmodel.display_name} version {dmodel.model_version_id} as it has no traffic.")

Model 05b_fraud has traffic = 100
Undeploying Endpoint model: projects/1026793852137/locations/us-central1/endpoints/1329608625143939072
Undeploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/1329608625143939072/operations/2863788329302753280
Endpoint model undeployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/1329608625143939072
Undeployed 05a_fraud version 1 as it has no traffic.


In [52]:
endpoint.traffic_split

{'1197082289624842240': 100}

In [53]:
endpoint.list_models()

[id: "1197082289624842240"
 model: "projects/1026793852137/locations/us-central1/models/model_05b_fraud"
 display_name: "05b_fraud"
 create_time {
   seconds: 1661005761
   nanos: 713426000
 }
 dedicated_resources {
   machine_spec {
     machine_type: "n1-standard-4"
   }
   min_replica_count: 1
   max_replica_count: 1
 }
 model_version_id: "1"]

---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [54]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()

In [55]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,35337,1.092844,-0.013230,1.359829,2.731537,-0.707357,0.873837,-0.796130,0.437707,0.396770,...,-0.167647,0.027557,0.592115,0.219695,0.036970,0.010984,0.0,0,a1b10547-d270-48c0-b902-7a0f735dadc7,TEST
1,60481,1.238973,0.035226,0.063003,0.641406,-0.260893,-0.580097,0.049938,-0.034733,0.405932,...,-0.057718,0.104983,0.537987,0.589563,-0.046207,-0.006212,0.0,0,814c62c8-ade4-47d5-bf83-313b0aafdee5,TEST
2,139587,1.870539,0.211079,0.224457,3.889486,-0.380177,0.249799,-0.577133,0.179189,-0.120462,...,0.180776,-0.060226,-0.228979,0.080827,0.009868,-0.036997,0.0,0,d08a1bfa-85c5-4f1b-9537-1c5a93e6afd0,TEST
3,162908,-3.368339,-1.980442,0.153645,-0.159795,3.847169,-3.516873,-1.209398,-0.292122,0.760543,...,-1.171627,0.214333,-0.159652,-0.060883,1.294977,0.120503,0.0,0,802f3307-8e5a-4475-b795-5d5d8d7d0120,TEST


In [56]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')[0]
#newob

In [57]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [58]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[[0.999369681, 0.000630355556]], deployed_model_id='1197082289624842240', model_version_id='1', model_resource_name='projects/1026793852137/locations/us-central1/models/model_05b_fraud', explanations=None)

In [59]:
prediction.predictions[0]

[0.999369681, 0.000630355556]

In [60]:
np.argmax(prediction.predictions[0])

0

### Get Predictions: REST

In [61]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [62]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    [
      0.999369681,
      0.000630355556
    ]
  ],
  "deployedModelId": "1197082289624842240",
  "model": "projects/1026793852137/locations/us-central1/models/model_05b_fraud",
  "modelDisplayName": "05b_fraud",
  "modelVersionId": "1"
}


### Get Predictions: gcloud (CLI)

In [63]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[0.999369681, 0.000630355556]]


---
## Remove Resources
see notebook "99 - Cleanup"